<center><h2>Подготовка работы в Colab</h2></center>

In [18]:
'''
WARNING
If you use google colab then run this cell and choose caps_net.py
file to upload to the colab server
otherwise ignore this cell!!!
'''
!pip install tqdm
from google.colab import files
src = list(files.upload().values())[0]
open('sources.zip','wb').write(src)

Saving sources.zip to sources (3).zip


8488758

In [0]:
import zipfile
import os
newpath = r'./data/sources' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
zip_ref = zipfile.ZipFile('sources.zip', 'r')
zip_ref.extractall('./data/sources/')
zip_ref.close()

In [20]:
!ls

data	     sources (1).zip  sources (3).zip
sample_data  sources (2).zip  sources.zip


<h2><center> Подготовка данных</center></h2>

In [0]:
from os import listdir, path
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [2]:
SOURCE_PATH = './data/sources/'

'''
sources_ws - split by white spaces and \n
sources_ch - split by chars
'''
sources_ws = []
sources_ch = []
for file_name in tqdm(listdir(SOURCE_PATH)):
    with open(path.join(SOURCE_PATH, file_name), 'r', encoding="cp1252") as file:
        data=file.read()
        sources_ws.append(data.split())
        sources_ch.append([ord(char) for char in list(data)])

100%|██████████| 21941/21941 [00:02<00:00, 7599.18it/s]


In [3]:
print(len(sources_ws))
print(len(sources_ch))

21941
21941


In [4]:
lengths_ws = [len(data) for data in sources_ws]
lengths_ch = [len(data) for data in sources_ch]
max_l_ws = max(lengths_ws)
min_l_ws = min(lengths_ws)
max_l_ch = max(lengths_ch)
min_l_ch = min(lengths_ch)
print('Максимальная длина разбинения по пробелам: ', max_l_ws)
print('Минимальная длина разбинения по пробелам: ', min_l_ws)
print('Максимальная длина разбинения по символам: ', max_l_ch)
print('Минимальная длина разбинения по символам: ', min_l_ch)

Максимальная длина разбинения по пробелам:  289
Минимальная длина разбинения по пробелам:  1
Максимальная длина разбинения по символам:  1023
Минимальная длина разбинения по символам:  7


In [0]:
PAD_SYMBOL = ord(' ')
flat_sources_ch = [ch for source in sources_ch for ch in source]
flat_sources_ch.append(PAD_SYMBOL)
unique_ch = set(flat_sources_ch)
ch2index = {ch: i for (i, ch) in enumerate(unique_ch)}
index2ch = {i: ch for (i, ch) in enumerate(unique_ch)}

In [0]:
a = [ch for source in sources_ch for ch in source]
ch_encoder = OneHotEncoder()
ch_one_hot = ch_encoder.fit_transform(np.reshape(a, (-1, 1)))

In [0]:
ch_one_hot_sources = []
end_index = 0
for l in lengths_ch:
    ch_one_hot_sources.append(ch_one_hot[end_index:end_index+l])
    end_index += l

In [0]:
def get_batch(x, batch_size):
    x_batch = np.array([seq.toarray() for seq in x[:batch_size]])
    seq_lengths = [seq.shape[0] for seq in x_batch]
    return x_batch, seq_lengths

def next_batch(x, size):
    idx = np.arange(0 , len(x))
    np.random.shuffle(idx)
    idx = idx[:size]
    x_batch = np.array([x[i].toarray() for i in idx])
    seq_lengths = [seq.shape[0] for seq in x_batch]
    return x_batch, seq_lengths

def add_padding(batch, pad_size):
    for i in range(len(batch)):
        batch[i] = np.pad(batch[i], [(0, pad_size - len(batch[i])), (0,0)],
                          mode='constant', constant_values=0)
    return np.reshape(np.vstack(batch), (-1, batch[0].shape[0], batch[0].shape[1]))

In [0]:
tf.reset_default_graph()

ENCODER_LAYERS = [256, 256]
DECODER_LAYERS = [256]
N_LATENT = 256
VEC_DIM = len(unique_ch)

X = tf.placeholder(tf.float32, [None, None, VEC_DIM])
Y = tf.placeholder(tf.float32, [None, None, VEC_DIM])
seq_length = tf.placeholder(tf.int32, [None])

with tf.variable_scope('encoder'):
    encoder_layers = [tf.nn.rnn_cell.GRUCell(size) for size in ENCODER_LAYERS]
    encoder_layer_cell = tf.contrib.rnn.MultiRNNCell(encoder_layers)
    _, encoder_state = tf.nn.dynamic_rnn(cell=encoder_layer_cell,
                                         inputs=X,
                                         dtype=tf.float32,
                                         sequence_length = seq_length)


with tf.variable_scope('latent_layer'):
    mean = tf.contrib.layers.fully_connected(encoder_state, N_LATENT, activation_fn=None)
    std_dev = 0.5 * tf.contrib.layers.fully_connected(encoder_state, N_LATENT, activation_fn=None)
    epsilon = tf.random_normal(shape=[1, 4, 3], dtype=tf.float32)
#     epsilon = tf.reshape(epsilon, (1, N_LATENT))
#     epsilon = tf.random_normal(tf.stack([tf.shape(encoder_state)[0], N_LATENT]))
#     t = tf.multiply(epsilon, tf.exp(std_dev))
#     z  = mean + tf.multiply(epsilon, tf.exp(std_dev))
    z = mean + std_dev
    
with tf.variable_scope('decoder'):
    decoder_input_pad = tf.zeros((tf.shape(X)[0], tf.shape(X)[1]-1, ENCODER_LAYERS[-1]))
    decoder_input = tf.concat([tf.reshape(z[-1], (-1, 1, ENCODER_LAYERS[-1])), decoder_input_pad], axis=1)
    decoder_layers = [tf.nn.rnn_cell.GRUCell(size) for size in DECODER_LAYERS]
    decoder_layer_cell = tf.contrib.rnn.MultiRNNCell(decoder_layers)
    decoder_out, decoder_state = tf.nn.dynamic_rnn(cell=decoder_layer_cell,
                                                   inputs=decoder_input,
                                                   dtype=tf.float32,
                                                   sequence_length = seq_length)

with tf.variable_scope('fc'):
    stacked_rnn_outputs = tf.reshape(decoder_out, (-1, DECODER_LAYERS[-1]))
    fc= tf.contrib.layers.fully_connected(stacked_rnn_outputs, VEC_DIM, activation_fn=None)
    logit = tf.reshape(fc, [-1, tf.shape(X)[1], VEC_DIM])
    
    target = tf.argmax(Y, axis=2)
    target_weights = tf.cast(tf.sequence_mask(seq_length, tf.reduce_max(seq_length)), tf.float32)
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(Y, logit))))
    target_loss = tf.contrib.seq2seq.sequence_loss(logit, target, target_weights)
#     kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     latent_loss = -0.5 * tf.reduce_sum(1.0 + 2.0 * sd - tf.square(mn) - tf.exp(2.0 * sd), 1)
    latent_loss = - 0.5 * tf.reduce_sum(1 + std_dev - tf.square(mean) - tf.exp(std_dev), axis=-1)
    loss = tf.reduce_mean(target_loss + latent_loss)

In [142]:
BATCH_SIZE = 1

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

x_batch, batch_seq_len = next_batch(ch_one_hot_sources, BATCH_SIZE)
print(batch_seq_len)
print(x_batch.shape)
# print(x_batch[:,:10,:].shape)
# print(x_batch[1].shape)
print('---=-=')
x_batch = add_padding(x_batch, max(batch_seq_len))
print(x_batch.shape)
x_batch = x_batch[:,:10,:]
batch_seq_len = [10]
print(x_batch.shape)
# train_x = np.array(ch_one_hot_sources[:2])
# dim = train_x[0].toarray()
# print(dim)
# print(train_x)
print('---')
print(sess.run(encoder_state, feed_dict={X: x_batch, seq_length: batch_seq_len})[-1].shape)
print(sess.run(epsilon, feed_dict={X: x_batch, seq_length: batch_seq_len})[-1].shape)
# print(sess.run(decoder_input_pad, feed_dict={X: x_batch, seq_length: batch_seq_len}))
# print(sess.run(decoder_input, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(decoder_out, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(logit, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(target, feed_dict={Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(target_weights, feed_dict={Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(target_loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(latent_loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))

# print(sess.run(t, feed_dict={X: x_batch, seq_length: batch_seq_len}))


[253]
(1, 253, 113)
---=-=
(1, 253, 113)
(1, 10, 113)
---
(1, 256)
(4, 3)


In [40]:
train_size = 0.99
X_train = ch_one_hot_sources[:int(train_size*len(ch_one_hot_sources))]
X_test = np.array([seq.toarray() for seq in ch_one_hot_sources[int(train_size*len(ch_one_hot_sources)):]])
print('Train set size: ', len(X_train))
print('Test set size: ', len(X_test))
test_seq_length = [seq.shape[0] for seq in X_test]
max_test_seq_length = max(test_seq_length)
X_test = add_padding(X_test, max_test_seq_length)

Train set size:  21721
Test set size:  220


In [88]:
LEARNING_RATE = 0.01
BATCH_SIZE = 50
EPOCHES = 10
N_BATHCES = int(len(X_train)/BATCH_SIZE)
print('Batches: ', N_BATHCES)

optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

for epoch in range(EPOCHES):
    print(X_test.shape)
    test_feed_dict = {
        X: X_test,
        Y: X_test,
        seq_length: test_seq_length
    }
    test_loss = sess.run(loss, feed_dict=test_feed_dict)
    print('Epoch %u, test loss: %f' %(epoch, test_loss))
    for i in range(N_BATHCES):
        x_batch, batch_seq_len = next_batch(X_train, BATCH_SIZE)
#         x_batch = x_batch[:, :10, :]
#         batch_seq_len = []
#         print(x_batch)
        x_batch = add_padding(x_batch, max(batch_seq_len))
        x_batch = x_batch[:, :10, :]
        batch_seq_len = [10] * x_batch.shape[0]
        feed_dict = {
            X: x_batch,
            Y: x_batch,
            seq_length: batch_seq_len
        }
        sess.run(training_op, feed_dict=feed_dict)
        if i % 10 == 0:
            train_loss = sess.run(loss, feed_dict=feed_dict)
            print(train_loss)

max_test_seq_length

Batches:  434
(220, 1019, 113)
Epoch 0, test loss: 4.727361
4.51234
2.0417645
1.9684969
2.031752
1.2441211
1.7821728
1.8279592
1.5516136
1.70524
1.578966
1.49376
1.5356264
1.2934256
1.3326514
1.84853
1.4050274
1.2200036
1.581924
1.3017532
0.91888195
1.167941
0.98204064
0.9825882
0.9957369
1.1812379
1.0036632
1.0388691
1.0996609
1.0321099
1.1118945
1.0904853
1.3949455
1.0252682


KeyboardInterrupt: ignored

In [0]:
x_batch